# Fast Prompting en Acción: Desentrañando la Magia
## Proyecto Final - Alex Quispe

### 📌 1. Introducción

En este proyecto se presenta una prueba de concepto (POC) que utiliza técnicas de **Fast Prompting** para resolver el problema de diseñar planes de ejercicio y alimentación personalizados.

**Problema:** Muchas personas buscan mejorar su salud pero enfrentan planes genéricos que no consideran sus limitaciones, tiempo ni preferencias.

**Solución:** Un asistente automatizado que genere rutinas de entrenamiento y alimentación personalizadas, además de material visual motivador.

### 📌 2. Configuración inicial

**Importante:** Antes de ejecutar este notebook, configura tu API Key de OpenAI como variable de entorno:

```bash
export OPENAI_API_KEY="tu_api_key_aqui"
```

O puedes configurarla directamente en el código (no recomendado para producción).

In [ ]:
# Instalación de dependencias (ejecutar solo si es necesario)
# !pip install openai

from openai import OpenAI
import os

# Configura tu API Key
client = OpenAI()
# Si no tienes la variable de entorno, descomenta y usa la siguiente línea:
# client = OpenAI(api_key="tu_api_key_aqui")

print("✅ Configuración completada")

### 📌 3. Prompts base para Fast Prompting

Definimos plantillas de prompts que usaremos en Fast Prompting. Se reemplazan `{campos}` con los datos del usuario.

In [ ]:
prompt_plan = """
Eres un entrenador personal virtual con conocimientos básicos de nutrición.
Recibirás los datos del usuario y debes generar:

1) Plan de entrenamiento semanal detallado.
2) Plan de comidas simple para 7 días.
3) Lista de compras agrupada por categorías.
4) Consejos de progresión y seguridad.

Datos del usuario:
Edad: {edad}
Sexo: {sexo}
Peso: {peso} kg
Altura: {altura} cm
Objetivo: {objetivo}
Nivel: {nivel}
Lesiones/limitaciones: {lesiones}
Disponibilidad: {dias_por_semana} días, {minutos_por_sesion} min/sesión
Equipo disponible: {equipo}
Preferencias alimentarias: {preferencias}

Entrega la respuesta estructurada en secciones numeradas.
"""

prompt_adaptacion = """
Toma el plan anterior y genera una versión alternativa que evite
impacto en {zona_lesionada}. Mantén la misma duración y objetivo,
proponiendo ejercicios sustitutos y modificaciones de intensidad.
"""

prompt_motivacion = """
Genera 7 mensajes motivacionales cortos, uno por día, orientados al
objetivo {objetivo}. Cada mensaje debe incluir un consejo práctico
y una micro-tarea sencilla.
"""

print("✅ Prompts definidos")

### 📌 4. Funciones auxiliares

In [ ]:
def generar_respuesta(prompt: str) -> str:
    """Llama a la API con un prompt y devuelve la respuesta."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=800
    )
    return response.choices[0].message.content

print("✅ Funciones auxiliares definidas")

### 📌 5. Caso de prueba con datos ficticios

Definimos un usuario de ejemplo para probar nuestro sistema de Fast Prompting.

In [ ]:
usuario = {
    "edad": 30,
    "sexo": "F",
    "peso": 72,
    "altura": 168,
    "objetivo": "pérdida de peso",
    "nivel": "principiante",
    "lesiones": "rodilla derecha (leve, sin cirugía)",
    "dias_por_semana": 4,
    "minutos_por_sesion": 45,
    "equipo": "banda elástica, mancuernas ligeras, silla",
    "preferencias": "vegetariana"
}

print("👤 Datos del usuario configurados:")
for key, value in usuario.items():
    print(f"  {key}: {value}")

### 📌 6. Generación del plan inicial

In [ ]:
# Generamos el plan inicial
prompt = prompt_plan.format(**usuario)
plan_inicial = generar_respuesta(prompt)

print("🏋️‍♀️ PLAN INICIAL GENERADO:")
print("=" * 50)
print(plan_inicial)

### 📌 7. Adaptación del plan con Fast Prompting

Ahora adaptamos el plan para tener en cuenta la lesión en la rodilla.

In [ ]:
prompt = prompt_adaptacion.format(zona_lesionada="rodilla derecha")
plan_adaptado = generar_respuesta(prompt)

print("🦵 PLAN ADAPTADO PARA LESIÓN:")
print("=" * 50)
print(plan_adaptado)

### 📌 8. Mensajes motivacionales

In [ ]:
prompt = prompt_motivacion.format(objetivo=usuario["objetivo"])
mensajes = generar_respuesta(prompt)

print("💪 MENSAJES MOTIVACIONALES:")
print("=" * 50)
print(mensajes)

### 📌 9. Experimentación con Fast Prompting

Ejemplo: pedimos al modelo que genere el plan en formato tabla en lugar de texto libre.

In [ ]:
prompt_tabla = prompt_plan.format(**usuario) + """
IMPORTANTE: Entrega los planes en formato de tabla Markdown para facilitar la lectura.
"""

plan_tabla = generar_respuesta(prompt_tabla)

print("📊 PLAN EN FORMATO TABLA:")
print("=" * 50)
print(plan_tabla)

### 10. Función para probar con nuevos usuarios

Creamos una función reutilizable para generar planes para cualquier usuario.

In [ ]:
def generar_plan_completo(datos_usuario, formato_tabla=False):
    """Genera un plan completo para un usuario."""
    
    # Elegir el prompt base
    prompt_base = prompt_plan
    if formato_tabla:
        prompt_base += "\nIMPORTANTE: Entrega los planes en formato de tabla Markdown."
    
    # Generar plan principal
    prompt = prompt_base.format(**datos_usuario)
    plan = generar_respuesta(prompt)
    
    # Generar mensajes motivacionales
    prompt_mot = prompt_motivacion.format(objetivo=datos_usuario["objetivo"])
    motivacion = generar_respuesta(prompt_mot)
    
    return {
        "plan_principal": plan,
        "motivacion": motivacion
    }

# Ejemplo de uso con nuevo usuario
nuevo_usuario = {
    "edad": 25,
    "sexo": "M",
    "peso": 80,
    "altura": 175,
    "objetivo": "ganancia de masa muscular",
    "nivel": "intermedio",
    "lesiones": "ninguna",
    "dias_por_semana": 5,
    "minutos_por_sesion": 60,
    "equipo": "gimnasio completo",
    "preferencias": "omnívoro"
}

print("🆕 PROBANDO CON NUEVO USUARIO:")
resultado = generar_plan_completo(nuevo_usuario, formato_tabla=True)
print("\n📋 PLAN:")
print(resultado["plan_principal"])
print("\n💪 MOTIVACIÓN:")
print(resultado["motivacion"])

### 11. Generación de Material Visual (Texto → Imagen)

Definimos los prompts para generación de imágenes

In [ ]:
prompts_imagen = {
    "tarjeta_sentadillas": """
    Create a vertical fitness instruction card (1080x1350) showing "Chair-assisted squats for knee-friendly workout". 
    Three numbered steps: 1) Woman standing behind sturdy chair, hands on backrest for support, 
    2) Slowly lowering into squat position while holding chair, keeping weight on heels, 
    3) Rising back to standing using chair for stability. Include "BEGINNER LEVEL" badge and 
    "KNEE-FRIENDLY" label. Clean minimalist style, light blue background, educational fitness illustration.
    """,
    
    "tarjeta_remo": """
    Create a vertical exercise card (1080x1350) for "Resistance band seated rows". 
    Three steps: 1) Woman sitting on floor, legs extended, resistance band around feet, arms forward, 
    2) Pulling band back with elbows close to ribs, squeezing shoulder blades, 
    3) Controlled return to starting position. "BEGINNER LEVEL" badge. 
    Clean instructional design, light background, fitness app style.
    """,
    
    "infografia_semanal": """
    Create a horizontal weekly workout infographic (1920x1080) titled "Sarah's 4-Day Beginner Workout Plan - Week 1". 
    Schedule: MONDAY: Full Body Strength (40 min), TUESDAY: Low-Impact Cardio (40 min), 
    WEDNESDAY: Rest Day, THURSDAY: Core & Stability (40 min), FRIDAY: Low-Impact HIIT (35 min), 
    WEEKEND: Rest Days. Include "KNEE-FRIENDLY MODIFICATIONS" badge, equipment needed, 
    total weekly time. Modern fitness app design, pastel colors, professional layout.
    """
}

Función para documentar la generación de imágenes

In [ ]:
def documentar_imagen_generada(nombre, herramienta, prompt_usado, archivo_resultado):
    """Documenta el proceso de generación de imágenes"""
    print(f"✅ {nombre}")
    print(f"   Herramienta utilizada: {herramienta}")
    print(f"   Dimensiones: {'1080x1350' if 'tarjeta' in archivo_resultado else '1920x1080'}")
    print(f"   Prompt utilizado:")
    print(f"   {prompt_usado[:100]}...")
    print(f"   Archivo generado: {archivo_resultado}")
    print(f"   Justificación: Se eligió {herramienta} por ser gratuita y generar imágenes de calidad educativa")
    print("-" * 50)

# Documentamos las imágenes que se generarían
print("\n📋 MATERIAL VISUAL A GENERAR:")
print("=" * 50)

In [ ]:
documentar_imagen_generada(
    "Tarjeta de Ejercicio: Sentadillas Asistidas con Silla",
    "NightCafe",
    prompts_imagen["tarjeta_sentadillas"],
    "sentadillas_asistidas.png"
)

# Tarjeta 2: Remo con banda
documentar_imagen_generada(
    "Tarjeta de Ejercicio: Remo con Banda Elástica",
    "NightCafe",
    prompts_imagen["tarjeta_remo"],
    "remo_banda_elastica.png"
)

# Infografía semanal
documentar_imagen_generada(
    "Infografía Semanal de Entrenamiento",
    "Leonardo AI",
    prompts_imagen["infografia_semanal"],
    "infografia_semanal.png"
)

print("\n🔄 PROCESO DE FAST PROMPTING PARA IMÁGENES:")
print("=" * 50)
print("1. OPTIMIZACIÓN: Los prompts incluyen dimensiones específicas y estilo deseado")
print("2. PERSONALIZACIÓN: Adaptados al usuario ejemplo (mujer, principiante, lesión rodilla)")
print("3. CONSISTENCIA: Mismo estilo visual para todas las tarjetas")
print("4. FUNCIONALIDAD: Diseño educativo e instructivo")

Función para generar prompt dinámico de imagen

In [ ]:
def generar_prompt_tarjeta(ejercicio, nivel, limitacion=None):
    """Genera prompts dinámicos para tarjetas de ejercicio"""
    
    base_prompt = f"""
    Create a vertical fitness instruction card (1080x1350) showing "{ejercicio}". 
    Three numbered steps showing proper form. Include "{nivel.upper()} LEVEL" badge. 
    Clean minimalist style, educational fitness illustration.
    """
    
    if limitacion:
        base_prompt += f' Include "{limitacion.upper()}" modification label.'
    
    return base_prompt.strip()

print("\n🚀 EJEMPLO DE GENERACIÓN DINÁMICA:")
print("=" * 50)

nuevo_prompt = generar_prompt_tarjeta(
    ejercicio="Wall Push-ups", 
    nivel="beginner",
    limitacion="knee-friendly"
)

print("Prompt generado dinámicamente:")
print(nuevo_prompt)

print("\n💡 TÉCNICAS DE FAST PROMPTING APLICADAS:")
print("=" * 50)
print("✅ Dimensiones específicas incluidas en el prompt")
print("✅ Estilo consistente definido ('minimalist', 'educational')")
print("✅ Elementos requeridos especificados ('3 steps', 'level badge')")
print("✅ Contexto del usuario integrado ('knee-friendly', 'beginner')")
print("✅ Generación batch de múltiples imágenes con plantillas")

print("\n📊 RESULTADOS ESPERADOS:")
print("=" * 50)
print("- Tarjetas de ejercicio claras e instructivas")
print("- Infografía semanal profesional y legible")  
print("- Consistencia visual en todo el material")
print("- Adaptación específica al perfil del usuario")
print("- Material listo para usar en la aplicación")


Simulación de resultados (para la documentación)

In [ ]:
resultados_simulados = {
    "tarjetas_generadas": 2,
    "infografias_generadas": 1, 
    "tiempo_total_generacion": "15 minutos",
    "herramientas_utilizadas": ["NightCafe", "Leonardo AI"],
    "calidad_obtenida": "Alta - apta para uso educativo"
}

print(f"\n📈 MÉTRICAS DEL PROCESO:")
print("=" * 50)
for metric, value in resultados_simulados.items():
    print(f"{metric.replace('_', ' ').title()}: {value}")


### 12. Conclusiones

**Logros del proyecto:**
- ✅ Usando Fast Prompting pudimos optimizar la salida (tablas, secciones claras)
- ✅ Se redujo el número de consultas al agrupar varias instrucciones en un solo prompt
- ✅ El sistema permite personalización rápida para distintos usuarios
- ✅ Implementación modular y reutilizable

**Limitaciones identificadas:**
- Costo de API por consulta
- Calidad de imágenes (requiere integración externa)
- Límite de tokens por respuesta

---

**¡Notebook completado!** 🎉

Para usar este notebook:
1. Configura tu API Key de OpenAI
2. Ejecuta todas las celdas en orden
3. Modifica los datos de usuario según tus necesidades
4. Experimenta con diferentes prompts y configuraciones